In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lime.lime_tabular import LimeTabularExplainer

import argparse
import datetime
import random
import signal
import sys
import warnings
from collections import Counter
from functools import partial
from pathlib import Path
from types import FrameType
from typing import Any

import dill
import multiprocess
import numpy as np
import pandas as pd
import torch
from synth_xai.explanations.explanation_utils import (
    evaluate_bb,
    find_top_closest_rows,
    get_test_data,
    is_explainer_supported,
    label_synthetic_data,
    load_bb,
    load_synthetic_data,
    make_predictions,
    prepare_neighbours,
    setup_wandb,
    transform_input_data,
)
from loguru import logger
from multiprocess import Pool
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import (
    MinMaxScaler,
)

from synth_xai.bb_architectures import MultiClassModel, SimpleModel
from synth_xai.explanations.explainer_model import ExplainerModel
from synth_xai.utils import (
    prepare_adult,
    prepare_dutch,
    prepare_letter,
    prepare_shuttle, 
    prepare_covertype, 
    prepare_house16,
)
import torch

/home/lcorbucci/synth_xai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("/home/lcorbucci/synth_xai/artifacts/house16/explanations/dt_ctgan_150000_2500_50.pkl", "rb") as f:
    exp_1 = dill.load(f)

with open("/home/lcorbucci/synth_xai/artifacts/house16/explanations/dt_ctgan_150000_2500_52.pkl", "rb") as f:
    exp_2 = dill.load(f)

In [4]:
exp_1[0]

(['(P14p9 = 0.120948) > 0.08191400021314621',
  '(P27p4 = 0.011348) <= 0.035534000024199486',
  '(H2p2 = 0.071146) > 0.057501498609781265',
  '(P1 = 1556) > 1303.5',
  '(P11p4 = 0.342545) > 0.07796349748969078',
  'Leaf node 89 reached, prediction: 1.0'],
 1)

In [5]:
exp_2[0]

(['(P14p9 = 0.120948) > 0.0898665003478527',
  '(H40p4 = 0.8) > 0.4696744978427887',
  '(P27p4 = 0.011348) <= 0.046157000586390495',
  '(H2p2 = 0.071146) > 0.03668750077486038',
  '(H13p1 = 0.121212) > 0.11366499960422516',
  '(P11p4 = 0.342545) <= 0.43805699050426483',
  '(P27p4 = 0.011348) <= 0.03520500101149082',
  '(P16p2 = 0.777305) <= 0.7973635196685791',
  '(H2p2 = 0.071146) <= 0.312731996178627',
  '(H2p2 = 0.071146) > 0.04505950026214123',
  'Leaf node 199 reached, prediction: 1.0'],
 1)

# Lore

In [82]:
from lore_sa.dataset import TabularDataset

# from lore_sa.lorem import LOREM

In [ ]:
class MyModel:
    def __init__(self, model: torch.nn.Module, scaler) -> None:
        self.model = model
        self.scaler = scaler

    def predict(self, x: np.ndarray) -> torch.Tensor:

        x = self.scaler.transform(x)

        predictions = []
        for sample in x:
            sample = torch.Tensor(sample)
            predictions.append(self.model(sample).argmax().item())
        return np.array(predictions)
        # print(x)
        # x = torch.Tensor(x)
        # print(x)
        # return np.array([self.model(sample).argmax().item() for sample in x])
    

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_covertype(sweep=False, seed=seed)#, current_path=current_script_path)


/home/lcorbucci/synth_xai/src/synth_xai/comparison


2025-03-12 10:51:12,075 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /datasets/mstz/covertype/resolve/main/README.md HTTP/1.1" 200 0
2025-03-12 10:51:12,077 urllib3.connectionpool DEBUG    Resetting dropped connection: s3.amazonaws.com
2025-03-12 10:51:12,420 urllib3.connectionpool DEBUG    https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/mstz/covertype/mstz/covertype.py HTTP/1.1" 404 0
2025-03-12 10:51:12,578 urllib3.connectionpool DEBUG    https://datasets-server.huggingface.co:443 "GET /parquet?dataset=mstz/covertype HTTP/1.1" 200 None
2025-03-12 10:51:12,741 urllib3.connectionpool DEBUG    https://datasets-server.huggingface.co:443 "GET /info?dataset=mstz/covertype HTTP/1.1" 200 None
2025-03-12 10:51:12,940 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/datasets/mstz/covertype/revision/refs%2Fconvert%2Fparquet HTTP/1.1" 200 825
2025-03-12 10:51:13,071 urllib3.connectionpool DEBUG    https://huggingface.co:443 "G

KeyError: "['class'] not found in axis"

In [84]:

target = "cover_type"
dataset = "covertype"
# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=[target]).columns)

bb_path = f"../../../artifacts/{dataset}/bb/{dataset}_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")


MultiClassModel(
  (layer1): Linear(in_features=54, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=7, bias=True)
)

In [85]:
import copy 

train_df_tmp = copy.copy(train_df)
train_df_tmp = train_df_tmp.drop(columns=[target])

scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df_tmp)

In [88]:
model = MyModel(model=bb, scaler=scaler)

num_classes = 2

In [89]:
train_df[target] = [int(x) for x in train_df[target]]

In [90]:
train_df[target] = train_df[target].astype('category')

In [91]:
dataset = TabularDataset.from_dict(train_df, class_name = target)
dataset.df.dropna(inplace = True)
dataset.df

,elevation,aspect,slope,horizontal_distance_to_hydrology,vertical_distance_to_hydrology,horizontal_distance_to_roadways,hillshade_9am,hillshade_noon,hillshade_3pm,horizontal_distance_to_fire_points,...,soil_type_id_31,soil_type_id_32,soil_type_id_33,soil_type_id_34,soil_type_id_35,soil_type_id_36,soil_type_id_37,soil_type_id_38,soil_type_id_39,cover_type
229430,2878.0,145.0,22.0,60.0,10.0,1513.0,244.0,230.0,104.0,2642.0,...,False,False,False,False,False,False,False,False,False,1
572404,2499.0,67.0,26.0,319.0,155.0,618.0,235.0,178.0,59.0,787.0,...,False,False,False,False,False,False,False,False,False,1
570202,2566.0,19.0,25.0,319.0,123.0,964.0,192.0,178.0,112.0,997.0,...,False,False,False,False,False,False,False,False,False,1
55212,2727.0,41.0,10.0,488.0,39.0,1348.0,221.0,218.0,131.0,2783.0,...,False,False,False,False,False,False,False,False,False,1
535623,3257.0,114.0,17.0,42.0,10.0,721.0,247.0,220.0,96.0,1198.0,...,False,False,False,False,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248501,3034.0,243.0,6.0,240.0,49.0,1250.0,207.0,245.0,176.0,1998.0,...,True,False,False,False,False,False,False,False,False,1
203275,3275.0,82.0,7.0,430.0,23.0,6242.0,230.0,228.0,132.0,603.0,...,False,False,False,False,False,False,False,False,False,0
180452,3210.0,52.0,6.0,467.0,31.0,5799.0,223.0,227.0,140.0,2753.0,...,False,False,False,False,False,False,False,False,False,1
528868,2933.0,122.0,27.0,85.0,32.0,2704.0,253.0,207.0,62.0,1717.0,...,False,False,False,False,False,False,False,False,False,0


In [92]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'ordinal', 'target'])

In [93]:
print("numeric: ", len(dataset.descriptor["numeric"]))
print("categorical: ", len(dataset.descriptor["categorical"]))
print("ordinal: ", len(dataset.descriptor["ordinal"]))

numeric:  10
categorical:  44
ordinal:  0


In [ ]:
from lore_sa.bbox import sklearn_classifier_bbox

In [ ]:
bbox = sklearn_classifier_bbox.sklearnBBox(model)

In [ ]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [ ]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
x

In [ ]:
# for i in range(0, 100):
#     x = dataset.df.iloc[i][:-1] # we exclude the target feature
#     print(bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax())

In [ ]:
num_rows = 100
multi_x = dataset.df.iloc[:num_row] # we exclude the target feature
multi_x = multi_x.drop(columns = ["income_binary"])

In [ ]:
bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax()

In [ ]:
model.predict(multi_x)

In [ ]:
dataset.descriptor.keys()

In [ ]:
dataset.descriptor["categorical"]

In [ ]:
dataset.descriptor["target"]

In [ ]:
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

In [ ]:
for premise in explanation["rule"]['premises']:
    print(premise["attr"])

In [ ]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lore/lore_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[1]

In [ ]:
shap_explanation[1]["rule"]["premises"]

# Lore - Dutch

In [ ]:
from lore_sa.dataset import TabularDataset

# from lore_sa.lorem import LOREM

In [ ]:
!export CUDA_VISIBLE_DEVICES=1

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [ ]:
class MyModel:
    def __init__(self, model: torch.nn.Module, scaler) -> None:
        self.model = model
        self.scaler = scaler

    def predict(self, x: np.ndarray) -> torch.Tensor:

        x = self.scaler.transform(x)

        predictions = []
        for sample in x:
            sample = torch.Tensor(sample)
            predictions.append(self.model(sample).argmax().item())
        return np.array(predictions)
        # print(x)
        # x = torch.Tensor(x)
        # print(x)
        # return np.array([self.model(sample).argmax().item() for sample in x])
    

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_dutch(sweep=False, seed=seed, current_path=current_script_path)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["occupation_binary"]).columns)

bb_path = "../../../artifacts/dutch/bb/dutch_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")

In [ ]:
import copy 

train_df_tmp = copy.copy(train_df)
train_df_tmp = train_df_tmp.drop(columns=["occupation_binary"])

scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df_tmp)

In [ ]:
model = MyModel(model=bb, scaler=scaler)

num_classes = 2

In [ ]:
train_df["occupation_binary"] = [int(x) for x in train_df["occupation_binary"]]

In [ ]:
train_df["occupation_binary"] = train_df["occupation_binary"].astype('category')    

In [ ]:
dataset = TabularDataset.from_dict(train_df, class_name = "occupation_binary")
dataset.df.dropna(inplace = True)
dataset.df

In [ ]:
dataset.descriptor.keys()

In [ ]:
dataset.descriptor["numeric"]

In [ ]:
from lore_sa.bbox import sklearn_classifier_bbox

In [ ]:
bbox = sklearn_classifier_bbox.sklearnBBox(model)

In [ ]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [ ]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
x

In [ ]:
# for i in range(0, 100):
#     x = dataset.df.iloc[i][:-1] # we exclude the target feature
#     print(bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax())

In [ ]:
num_rows = 100
multi_x = dataset.df.iloc[:num_row] # we exclude the target feature
multi_x = multi_x.drop(columns = ["income_binary"])

In [ ]:
bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax()

In [ ]:
model.predict(multi_x)

In [ ]:
dataset.descriptor.keys()

In [ ]:
dataset.descriptor["target"]

In [ ]:
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

In [ ]:
for premise in explanation["rule"]['premises']:
    print(premise["attr"])

In [ ]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lore/lore_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[1]

In [ ]:
shap_explanation[1]["rule"]["premises"]

# Lore Multiclass

In [ ]:
from lore_sa.dataset import TabularDataset

# from lore_sa.lorem import LOREM

In [ ]:
class MyModel:
    def __init__(self, model: torch.nn.Module, scaler) -> None:
        self.model = model
        self.scaler = scaler

    def predict(self, x: np.ndarray) -> torch.Tensor:

        x = self.scaler.transform(x)

        predictions = []
        for sample in x:
            sample = torch.Tensor(sample)
            predictions.append(self.model(sample).argmax().item())
        return np.array(predictions)
        # print(x)
        # x = torch.Tensor(x)
        # print(x)
        # return np.array([self.model(sample).argmax().item() for sample in x])
    

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_letter(sweep=False, seed=seed)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["letter"]).columns)

bb_path = "../../../artifacts/letter/bb/letter_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")


In [ ]:
import copy 

train_df_tmp = copy.copy(train_df)
train_df_tmp = train_df_tmp.drop(columns=["letter"])

scaler = MinMaxScaler()
x_train = scaler.fit_transform(train_df_tmp)

In [ ]:
model = MyModel(model=bb, scaler=scaler)

num_classes = 2

In [ ]:
train_df["letter"] = [int(x) for x in train_df["letter"]]

In [ ]:
train_df["letter"] = train_df["letter"].astype('category')

In [ ]:
dataset = TabularDataset.from_dict(train_df, class_name = "letter")
dataset.df.dropna(inplace = True)
dataset.df

In [ ]:
dataset.descriptor.keys()

In [ ]:
from lore_sa.bbox import sklearn_classifier_bbox

In [ ]:
bbox = sklearn_classifier_bbox.sklearnBBox(model)

In [ ]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [ ]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
x

In [ ]:
# for i in range(0, 100):
#     x = dataset.df.iloc[i][:-1] # we exclude the target feature
#     print(bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax())

In [ ]:
num_rows = 100
multi_x = dataset.df.iloc[:num_row] # we exclude the target feature
multi_x = multi_x.drop(columns = ["letter"])

In [ ]:
bb(torch.Tensor(scaler.transform(np.array(x.values).reshape(1, -1)))).argmax()

In [ ]:
model.predict(multi_x)

In [ ]:
dataset.descriptor.keys()

In [ ]:
dataset.descriptor["target"]

In [ ]:
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

In [ ]:
for premise in explanation["rule"]['premises']:
    print(premise["attr"])

In [ ]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lore/lore_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[1]

# Load Shap explanation

In [ ]:
with open("/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/shap/shap_1.pkl", "rb") as f:
    shap_explanation = dill.load(f)

In [ ]:
shap_explanation[0][0]

In [ ]:
class aix_model:
    def __init__(self, model: torch.nn.Module) -> None:
        self.model = model

    def predict(self, x: np.ndarray) -> torch.Tensor:
        x = torch.Tensor(x)
        return self.model(x).argmax(dim=1)

    def predict_proba(self, x: np.ndarray) -> np.ndarray:
        # since the activation function of the last layer is LogSoftmax
        # we need to apply the exponential to the output of the model
        # cast x to be a Tensor
        x = torch.Tensor(x)
        return torch.nn.functional.softmax(self.model(x)).detach().numpy()


In [ ]:
seed = 112
current_script_path = Path("./").resolve()

x_train, _, x_test, _, _, _, train_df, test_data = prepare_adult(sweep=False, seed=seed, current_path=current_script_path)
scaler = MinMaxScaler()
_ = scaler.fit_transform(x_train)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["income_binary"]).columns)

# Identify categorical features (before one-hot encoding)
categorical_features = [
    train_df.columns.get_loc(col)
    for col in ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
    if col in train_df.columns
]

# Define the class names
class_names = ["<=50K", ">50K"]

In [62]:
categorical_features = []
feature_names = ["age","household_position","household_size","prev_residence_place","citizenship","country_birth","edu_level","economic_status", "cur_eco_activity","Marital_status","sex_binary", "occupation_binary"]
class_names = ["<=50K", ">50K"]

In [63]:
# Initialize LimeTabularExplainer
explainer = LimeTabularExplainer(
    x_train,  # Unscaled training data
    mode="classification",
    feature_names=feature_names,
    categorical_features=categorical_features,
    class_names=class_names,
    discretize_continuous=True,  # Discretize continuous features for better interpretability
)

In [ ]:
bb_path = "../../../artifacts/adult/bb/adult_BB.pth"
bb = load_bb(bb_path)

In [ ]:
bb.to("cpu")
model = aix_model(model=bb)
# model = bb

In [65]:
def predict_fn(x: np.ndarray) -> np.ndarray:
        model.model.to("cpu")
        prediction = model.predict_proba(x)
        return prediction

In [68]:
# Generate explanation
sample_idx = 51  # Choose any index from x_test
sample = np.array(x_test[sample_idx])
explanation = explainer.explain_instance(sample, predict_fn, num_features=10)
explanation.as_list()
explanation.local_pred

array([0.46324448])

In [69]:
explanation.as_list()

[('0.60 < edu_level <= 1.00', -0.4550781982046549),
 ('sex_binary <= 0.00', 0.27073169153941756),
 ('age <= 0.27', 0.09339496829436184),
 ('prev_residence_place <= 0.00', 0.07893404544743478),
 ('country_birth <= 0.00', -0.05718273197171243),
 ('citizenship <= 0.00', -0.041228587391163876),
 ('Marital_status <= 0.00', 0.028421249345087073),
 ('0.11 < household_position <= 0.19', -0.01062319708006088),
 ('cur_eco_activity > 0.93', -0.009474145521056973),
 ('household_size <= 0.07', 0.004768681924036221)]

In [ ]:
explanation.score

In [ ]:
import re 
feature_names = [feature for feature, weight in explanation.as_list()]
clean_features = [re.sub(r'[<>]=?|\d+(\.\d+)?', '', feature).strip() for feature in feature_names]

clean_features

In [ ]:
store_path = "/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lime/"
file_name = "lime_1.pkl"
store_path = Path(store_path) / file_name
with Path(store_path).open("rb") as f:
    explanation = dill.load(f)


In [ ]:
explanation[45][2]

In [ ]:
store_path = "/home/lcorbucci/synth_xai/artifacts/adult/comparison_explanation/lime/"
file_name = "lime_2.pkl"
store_path = Path(store_path) / file_name
with Path(store_path).open("rb") as f:
    explanation = dill.load(f)


In [ ]:
explanation[45][2]

In [ ]:
store_path = "/home/lcorbucci/synth_xai/artifacts/adult/explanations/"
file_name = "dt_tvae_100000_2500_1.pkl"
store_path = Path(store_path) / file_name
with Path(store_path).open("rb") as f:
    explanation = dill.load(f)


# Shap

In [2]:
from pathlib import Path

import numpy as np
import torch
import shap

from synth_xai.bb_architectures import MultiClassModel, SimpleModel
from synth_xai.explanations.explanation_utils import load_bb
from synth_xai.utils import prepare_adult
from sklearn.cluster import KMeans

In [3]:
np.empty((0,2))

array([], shape=(0, 2), dtype=float64)

In [ ]:
# def get_representative_samples(dataset,samples_per_class=5):
#     '''this function is used to get the representative samples of the dataset. but in a different way.
#     First there is a clustering of all the data. The best k is retrieved using the elbow method.
#     Then, for each cluster we order the samples by the distance from the centroid in 4 bins.
#     Then we take some samples of each bin as the representative sample of the cluster, (in order to have the label)'''
#     # print(dataset.tensors[1].shape,"shape of the labels")
#     num_classes = 2
#     representative_samples = np.empty((0,dataset.shape[1]))
#     labels = np.empty((0,num_classes))
#     # create the kmeans object
#     inertias = []
#     max_K = min(12,dataset.shape[0])
#     for i in range(2,max_K):
#         kmeans = KMeans(n_clusters=i, random_state=42,n_init=3)
#         kmeans.fit(dataset)
#         # get the centroids of the clusters
#         inertias.append(kmeans.inertia_)

#     # coupute the best k
#     inertias_diff = np.diff(inertias)
#     # find the last highest difference
#     k = inertias_diff[::-1].argmax()
#     k = len(inertias) - k
#     print("best k:",k)
#     # plot the elbow plot

#     kmeans = KMeans(n_clusters=k, random_state=42,n_init=3)
#     kmeans.fit(dataset)
#     # get the centroids of the clusters
#     centroids = kmeans.cluster_centers_
#     # get the samples that are mapped to each cluster
#     labels = kmeans.labels_
#     representative_labels = np.empty((0,num_classes))
#     for cluster in range(k):
#         # get the samples of the cluster
#         cluster_samples = dataset[labels==cluster,:]
#         # get the distances of the samples from the centroid
#         distances = np.linalg.norm(cluster_samples-centroids[cluster],axis=1)
#         # order the samples in 4 bins using again KMeans, but on the distances
#         kkkk = min(4,cluster_samples.shape[0])
#         kmeans = KMeans(n_clusters=kkkk, random_state=42,n_init=3)
#         kmeans.fit(distances.reshape(-1,1))
#         ordered_samples = cluster_samples[np.argsort(kmeans.labels_)]

        
#         # divide the samples in 4 bins
#         bins = [ordered_samples[kmeans.labels_==i] for i in range(4)]
#         # plot the number of samples in each bin
    
#         original_labels_of_cluster = dataset[labels==cluster,:]
#         print(original_labels_of_cluster)
        
                                            
#         # take the first sample of each bin, and its relative label
#         for d_i in range(4):
#             # if there are no samples in the bin, skip it
#             if len(bins[d_i]) == 0:
#                 continue
#             # choose a random sample from the bin
#             rnd_sample = 0#np.random.choice(bins[d_i])
#             representative_samples = np.vstack((representative_samples,bins[d_i][rnd_sample]))
#             print(original_labels_of_cluster[rnd_sample,:])
#             print(representative_labels)
#             representative_labels = np.vstack([representative_labels,
#                                                 original_labels_of_cluster[rnd_sample,:]])
                                    
#     print(representative_samples.shape,representative_labels.shape)
#     return representative_samples,representative_labels

In [ ]:
# feats, labs = get_representative_samples(x_train[0:40], samples_per_class=30//num_classes)

In [5]:
class aix_model:
    def __init__(self, model: torch.nn.Module) -> None:
        self.model = model

    def predict(self, x: np.ndarray) -> torch.Tensor:
        x = torch.Tensor(x)
        return self.model(x).argmax(dim=1)

    def predict_proba(self, x: np.ndarray) -> np.ndarray:
        # since the activation function of the last layer is LogSoftmax
        # we need to apply the exponential to the output of the model
        x = torch.Tensor(x)
        return torch.nn.functional.softmax(self.model(x), dim=1).detach().numpy()

seed = 112
current_script_path = Path("./").resolve()
print(current_script_path)
x_train, _, x_test, _, _, _, train_df, test_data = prepare_dutch(sweep=False, seed=seed, current_path=current_script_path)

# Get the feature names after one-hot encoding
feature_names = list(train_df.drop(columns=["occupation_binary"]).columns)

bb_path = "../../../artifacts/dutch/bb/dutch_BB.pth"
bb = load_bb(bb_path)

bb.to("cpu")
model = aix_model(model=bb)

num_classes = 2

/home/lcorbucci/synth_xai/src/synth_xai/comparison


In [6]:
# Define SHAP explainer
# explainer = shap.Explainer(model.predict_proba, x_test)
explainer = shap.KernelExplainer(model.predict_proba, data=shap.kmeans(x_test, 20))

In [74]:
sample_idx = 51

In [75]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

sample = test_data.iloc[sample_idx]

# Get the feature values of the sample (excluding target column)
sample_features = sample.drop('occupation_binary').values.reshape(1, -1)

# Get all other samples (excluding target column)
other_samples = test_data.drop(columns=['occupation_binary'])

# Calculate cosine similarity
similarities = cosine_similarity(sample_features, other_samples)

# Get indices of top 3 most similar samples (excluding itself if present)
top_similar_indices = np.argsort(-similarities[0])[:4]  # Get 4 in case the sample itself is included
top_similar_indices = [idx for idx in top_similar_indices if test_data.index[idx] != sample.name][:3]

# Display the top 3 most similar samples
print("Sample:")
print(sample)
print("\nTop 3 most similar samples:")
for i, idx in enumerate(top_similar_indices):
    similar_sample = test_data.iloc[idx]
    similarity = similarities[0][idx]
    print(f"\nSimilar sample {i+1} (similarity: {similarity:.4f}):")
    print(similar_sample)


Sample:
age                        7
household_position      1131
household_size           112
prev_residence_place       1
citizenship                1
country_birth              1
edu_level                  5
economic_status          111
cur_eco_activity         138
Marital_status             1
sex_binary                 0
occupation_binary          0
Name: 2229, dtype: int64

Top 3 most similar samples:

Similar sample 1 (similarity: 1.0000):
age                        7
household_position      1131
household_size           112
prev_residence_place       1
citizenship                1
country_birth              1
edu_level                  5
economic_status          111
cur_eco_activity         138
Marital_status             1
sex_binary                 0
occupation_binary          0
Name: 51705, dtype: int64

Similar sample 2 (similarity: 1.0000):
age                        7
household_position      1131
household_size           112
prev_residence_place       1
citizenship         

In [50]:
test_data.iloc[sample_idx]

age                        7
household_position      1131
household_size           112
prev_residence_place       1
citizenship                1
country_birth              1
edu_level                  5
economic_status          111
cur_eco_activity         138
Marital_status             1
sex_binary                 0
occupation_binary          0
Name: 2229, dtype: int64

In [51]:

# Generate explanation
 # Choose any index from x_test
sample = x_test[sample_idx:sample_idx+1]  # Keep sample as a batch
shap_values = explainer(sample)

100%|██████████| 1/1 [00:00<00:00,  4.63it/s]


In [52]:
sample

array([[0.27272727, 0.19090909, 0.06666667, 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.96428571, 0.        ,
        0.        ]])

In [53]:
predicted_class = model.predict(sample)
predicted_class = predicted_class.item()

In [54]:
predicted_class

0

In [55]:
model.predict_proba(sample)

array([[0.86464745, 0.13535258]], dtype=float32)

In [45]:
# shap.plots.watearfall(shap_values[0,:,predicted_class])

In [56]:
shap_values.values[0,:,predicted_class]

array([-0.00782544,  0.00422991, -0.00202537, -0.00261117,  0.        ,
        0.00448547,  0.47456479, -0.01099995,  0.04268176, -0.01348994,
       -0.09987371])

In [57]:
feature_importance = shap_values[0,:,predicted_class].values

In [58]:
feature_importance

array([-0.00782544,  0.00422991, -0.00202537, -0.00261117,  0.        ,
        0.00448547,  0.47456479, -0.01099995,  0.04268176, -0.01348994,
       -0.09987371])

In [ ]:
list(zip(feature_names, feature_importance))

In [ ]:
# get the feature names


# Simple example with Adult

In [ ]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

In [ ]:
feature_names = ["Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Marital Status","Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss","Hours per week", "Country"]

In [ ]:
data = np.genfromtxt('./adult.data', delimiter=', ', dtype=str)

In [ ]:
labels = data[:,14]
le= sklearn.preprocessing.LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
class_names = le.classes_
data = data[:,:-1]

In [ ]:
categorical_features = [1,3,5, 6,7,8,9,13]

In [ ]:
categorical_names = {}
for feature in categorical_features:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    categorical_names[feature] = le.classes_


In [ ]:
data = data.astype(float)

In [ ]:
from sklearn.compose import ColumnTransformer

encoder = ColumnTransformer(
	transformers=[
		('cat', sklearn.preprocessing.OneHotEncoder(), categorical_features)
	],
	remainder='passthrough'
)

In [ ]:
np.random.seed(1)
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(data, labels, train_size=0.80)


In [ ]:
encoder.fit(data)
encoded_train = encoder.transform(train)

In [ ]:
import xgboost
gbtree = xgboost.XGBClassifier(n_estimators=300, max_depth=5)
gbtree.fit(encoded_train, labels_train)

In [ ]:
sklearn.metrics.accuracy_score(labels_test, gbtree.predict(encoder.transform(test)))

In [ ]:
predict_fn = lambda x: gbtree.predict_proba(encoder.transform(x)).astype(float)

In [ ]:
train

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train ,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)

In [ ]:
np.random.seed(1)
i = 1
exp = explainer.explain_instance(test[i], predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)

In [ ]:
test[i]

In [ ]:
exp.local_pred